In [1]:
import pandas as pd
import numpy as np
import pickle
import gzip
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
import yaml
import xgboost
from xgboost import XGBClassifier
with open("configuration.yaml", "r") as yml_file:
    config = yaml.load(yml_file, yaml.Loader)

COLUMNS_TRAINING = config['COLUMNS_TRAINING']

C:\Users\Sana\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\Sana\AppData\Local\Temp\ipykernel_29892\1585305739.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def sort_filter(x_train, x_test, y_train, y_test):
    """
    Final preparation steps to achieve a competent CV data:
    1) remove repeated data items in the test datasets
       removing duplicate sites in test dataset --but not in training-- as repeated data in training provide us with
       weight of evidence, whereas too many repeated data at testing can spoil our capacity to evaluate
    2) random sampling to get a balanced test set
    3) set training feature labels in a canonical order taken from configuration
    """

    # reset index

    x_test.reset_index(inplace=True, drop=True)
    y_test.reset_index(inplace=True, drop=True)

    # remove duplicates from test set

    x_test['chr'] = x_test['chr'].astype(str)
    x_test['start'] = x_test['start'].astype(int)
    x_test = x_test.drop_duplicates(['start', 'alt'])
    test_index = x_test.index
    y_test = y_test.loc[y_test.index.intersection(test_index)]

    # balance test set

    total_index = set(x_test.index)
    if y_test.mean() <= 0.5:
        balance_index = y_test[y_test == 1].index.tolist()
    else:
        balance_index = y_test[y_test == 0].index.tolist()
    remaining_index = list(set(total_index) - set(balance_index))
    balance_index += list(np.random.choice(remaining_index, size=len(balance_index), replace=False))
    x_test = x_test.loc[x_test.index.intersection(balance_index)]
    y_test = y_test.loc[y_test.index.intersection(balance_index)]

    # feature labels in standard order
    avoid = ['chr', 'start', 'ref', 'alt']
    features = list(filter(lambda x: x not in avoid, x_train))
    # print(set(features))
    # assert (set(features) == set(COLUMNS_TRAINING))
    x_train = x_train[avoid + COLUMNS_TRAINING]
    x_test = x_test[avoid + COLUMNS_TRAINING]

    return x_train, x_test, y_train, y_test

def split_balanced(x_data, y_data, test_size=0.3):
    """Generate balanced train-test split"""

    one_index  = list(y_data[y_data == 1].index)
    zero_index = list(y_data[y_data == 0].index)

    # randomly select n_ones indices from zero_index
    zero_index = list(np.random.choice(zero_index, size=len(one_index), replace=False))

    x_data_sub = x_data.loc[one_index + zero_index, :]
    y_data_sub = y_data.loc[one_index + zero_index]

    # the random state should be fixed prior to this call
    x_train, x_test, y_train, y_test = train_test_split(x_data_sub, y_data_sub, test_size=test_size)
    return x_train, x_test, y_train, y_test

def get_cv_sets_balanced(x_data, y_data, n, size):
    """Generate several balanced train-test sets"""

    for _ in range(n):
        x_train, x_test, y_train, y_test = split_balanced(x_data, y_data, test_size=size)
        yield x_train, x_test, y_train, y_test

def prepare(data, nsplits=10, test_size=0.2):

    data.rename(columns={'response': 'label'}, inplace=True)

    # keep 'pos' and 'chr' to run position-based filtering
    avoid = ['cohort', 'gene', 'aachange', 'label', 'motif']  # include 'ref' 'alt'
    features = list(filter(lambda x: x not in avoid, data.columns))

    # regression datasets
    x_data = data[features]
    y_data = data['driver'] # label

    # cv_list output has tuples (x_train, x_test, y_train, y_test) as elements
    cv_list = get_cv_sets_balanced(x_data, y_data, nsplits, test_size)

    # filter test data to prevent site repetitions
    cv_list = [sort_filter(*arg) for arg in cv_list]

    return cv_list

In [3]:
def load_mutations(input_path):

    data = pd.read_csv(input_path, sep=',', low_memory=False)
    return data

def generate(mutations, random_state=None, bootstrap_splits=50, cv_fraction=0.3):

    # fix random seed (if provided)
    np.random.seed(random_state)

    d_output = {}

    # for gene, data in mutations.groupby('gene'):
    cv_list = prepare(mutations.copy(), nsplits=bootstrap_splits, test_size=cv_fraction) # data.copy()
    d_output = cv_list

    return d_output

In [4]:
mutations = load_mutations('data/final_dataset.csv')

mutations.drop(['chr_y', 'start_y', 'end_y'], inplace=True, axis=1)
mutations.rename({'chr_x': 'chr', 'start_x': 'start', 'end_x': 'end'}, inplace=True, axis=1)

# d_output = generate(mutations,
#                     random_state=104,
#                     bootstrap_splits=50,
#                     cv_fraction=0.3)

# with gzip.open('data/splits.pkl', 'wb') as f:
#     pickle.dump(d_output, f)

# Training

In [5]:
dict_results = {
        'models': [], 'split_number': [], 'x_test': [], 'y_test': [], 'learning_curves': []
    }

In [7]:
def train(values):
    """Returns: optimal classification threshold and trained XGB model"""

    x_train, x_test, y_train, y_test, split_number, seed = tuple(values)
    XGB_PARAMS = config['XGB_PARAMS']
    params = XGB_PARAMS.copy()
    params['n_estimators'] = 20000  # set it high enough to allow "early stopping" events below
    params['base_score'] = y_train.mean()
    params['n_jobs'] = 1
    params['seed'] = seed
    myclassifier = XGBClassifier(**params)

    # train with xgboost
    learning_curve_dict = {}
    myclassifier.train(x_train, y_train,
                       eval_set=[(x_train, y_train), (x_test, y_test)],
                       eval_metric='logloss',  # mcc_loss could be used here
                       early_stopping_rounds=2000,
                       callbacks=[
                           xgboost.callback.EvaluationMonitor(rank=0, period=1, show_stdv=False),
                       ],
                       verbose=False)

    params['n_estimators'] = myclassifier.model.best_iteration
    learning_curve_dict = {k: v['logloss'][:params['n_estimators']] for k, v in learning_curve_dict.items()}
    myclassifier.model.set_params(**params)

    return myclassifier, split_number, x_test, y_test, learning_curve_dict

file_cv = 'data/splits.pkl'
min_rows = 30   # Minimum number of rows to carry out training
non_features = ['start', 'chr', 'ref', 'alt']
cores = 1
with Pool(cores) as p:
    with gzip.open(file_cv, 'rb') as f:
        split_cv = pickle.load(f)

    mean_size = np.nanmean([cv[0].shape[0] for cv in split_cv])
    if mean_size < min_rows:
        print("ERROR")

    list_cvs = []
    for i, x in enumerate(split_cv):
        x_list = list(x) + [i, np.random.randint(100000)]

        # filter out non-features, i.e., columns not used for training
        x_list[0] = x_list[0].drop(non_features, axis=1)
        x_list[1] = x_list[1].drop(non_features, axis=1)
        list_cvs.append(x_list)

    for model, split_number, x_test, y_test, learning_curve in p.imap(
            train, list_cvs):
        dict_results['models'].append(model)
        dict_results['split_number'].append(split_number)
        dict_results['x_test'].append(x_test)
        dict_results['y_test'].append(y_test)
        dict_results['learning_curves'].append(learning_curve)

    with gzip.open('data/models.pkl', 'wb') as f:
        pickle.dump(dict_results, f)

In [ ]:
import functools
COLUMNS_SHAP = [f'my_shap_{x}' for x in COLUMNS_TRAINING]
def predict(mutations, ttype, models, models_folder, evaluations_folder):

    # hierarchy = Hierarchy(models)

    # print(ttype)
    # print(mutations['gene'].unique())
    # get_model = functools.partial(hierarchy.get_model_for_gene, ttype=ttype)
    get_model = models
    d = dict(map(get_model, mutations['gene'].unique()))
    mutations['selected_model_ttype'], mutations['selected_model_gene'] = zip(*mutations['gene'].map(d))

    return _predict(mutations, models_folder, evaluations_folder)

def _predict(mutations, models_folder, evaluations_folder):

    # Prepare space for the columns
    for c in COLUMNS_SHAP:
        mutations[c] = np.nan
    mutations['boostDM_score'] = np.nan

    for name, df in mutations.groupby(['selected_model_gene', 'selected_model_ttype']):
        gene, ttype = name
        print(gene, ttype)
        if gene is None:
            return
        else:
            path_model = os.path.join(models_folder, f'{ttype}', f'{gene}.models.pickle.gz')
            with gzip.open(path_model, 'rb') as f:
                model = pickle.load(f)

            path_eval = os.path.join(evaluations_folder, f'{ttype}', f'{gene}.eval.pickle.gz')
            with gzip.open(path_eval, 'rb') as g:
                evaluation = pickle.load(g)

            mutations.loc[df.index, ['boostDM_score']] = bootstrap_voting(model, evaluation)(df[COLUMNS_TRAINING])

            # SHAP attribution
            x_data = df[COLUMNS_TRAINING]

            shap_bootstrap = []
            for model in model['models']:
                explainer = shap.TreeExplainer(model.model)
                shap_bootstrap.append(explainer.shap_values(x_data))
            shap_values = np.mean(shap_bootstrap, axis=0)

            mutations.loc[df.index, COLUMNS_SHAP] = shap_values

    mutations.loc[:, 'boostDM_class'] = mutations['boostDM_score'].apply(lambda x: x >= 0.5)
    return mutations

def bootstrap_voting(model_obj, eval_obj, use_weights=True):
    """
    Args:
        model_obj: instance of model for a specific ttype-gene
        eval_obj: evaluation instance for the same specific ttype-gene
        use_weights
    Return:
        function: that given a vector of features, it returns a prediction label and score
                  based on individual predictions and voting among all models and thresholds
        warning:  at this point probabilities may need a final recalibration step
    Satopaa, V. A. et al. Combining multiple probability predictions using a simple logit model.
    International Journal of Forecasting 30, 344–356 (2014).
    """

    models = model_obj['models']
    logloss = eval_obj['logloss']

    # estimate overall systematic bias
    weights = 1 / np.array(logloss)
    weights = weights / np.sum(weights)
    bias = 2.3  # estimate of systematic bias based on testing with BRCA TCGA data

    def func(x):

        prod = 1
        for i, model in enumerate(models):
            feature_names = model.model.get_booster().feature_names
            p = model.predict_proba(x[feature_names])[:, 1]
            if use_weights:
                prod *= (p / (1 - p)) ** weights[i]  # weighting based on logloss
            else:
                prod *= (p / (1 - p)) ** (1 / len(weights))  # otherwise, balanced weighting
        prod = prod ** bias  # correction for systematic bias
        s = (prod / (1 + prod))

        return s

    return func

In [8]:
with gzip.open("models.pickle.gz", 'rb') as g:
    models = pickle.load(g)
# models = pickle.load(open("models.pickle.gz", 'rb'))
# df = predict(mutations, models=models, models_folder="models_dir/", evaluations_folder="eval_dir/")

EOFError: Compressed file ended before the end-of-stream marker was reached

In [16]:
!pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB 1.3 MB/s eta 0:00:10
   ---------------------------------------- 0.1/11.6 MB 656.4 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.6 MB 1.2 MB/s eta 0:00:10
    --------------------------------------- 0.2/11.6 MB 1.5 MB/s eta 0:00:08
   - -------------------------------------- 0.3/11.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.4/11.6 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.4/11.6 MB 2.0 MB/s eta 0:00:06
   - -------------------------------------- 0.4/11.6 MB 2.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.0/11.6 MB 2.6 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.6 MB 2.4 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.6 MB 2.4 MB/s eta 0:00:05
   


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB 1.1 MB/s eta 0:01:30
   ---------------------------------------- 0.1/99.8 MB 991.0 kB/s eta 0:01:41
   ---------------------------------------- 0.2/99.8 MB 1.5 MB/s eta 0:01:05
   ---------------------------------------- 0.3/99.8 MB 1.6 MB/s eta 0:01:03
   ---------------------------------------- 0.5/99.8 MB 2.3 MB/s eta 0:00:45
   ---------------------------------------- 0.7/99.8 MB 2.4 MB/s eta 0:00:42
   ---------------------------------------- 0.7/99.8 MB 2.4 MB/s eta 0:00:42
   ---------------------------------------- 0.8/99.8 MB 2.3 MB/s eta 0:00:43
   ---------------------------------------- 0.8/99.8 MB 2.3 MB/s eta 0:00:43
   ---------------------------------------- 1.0/99.8 MB 2.2 MB/s eta 0:00:44
   -------


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
